In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch # pytorch library
from torch.autograd import Variable # variables may accumulate gradients
import torchvision.transforms as transforms
from torch.distributions import normal
from sklearn.model_selection import train_test_split
import torch.nn as nn
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# compare numpy and torch arrays/tensors
# numpy
array = [[1,2,3],[4,5,6]]
first_array = np.array(array) # 2x3 array
print("Array Type: {}".format(type(first_array))) # type
print("Array Shape: {}".format(np.shape(first_array))) # shape
print(first_array)
print()

# torch
tensor = torch.Tensor(array) # from python list
print("Array Type: {}".format(tensor.type)) # type
print("Array Shape: {}".format(tensor.shape)) # shape
print(tensor)

In [ ]:
# INITIALISATION

# numpy ones
print("Numpy\n {}\n".format(np.ones((2,3))))

# pytorch ones
print(torch.ones((2,3)))
print()

# numpy random
print("Numpy\n {}\n".format(np.random.rand(5,3)))

# pytorch random
print(torch.rand(5,3))
print()

# uninitialised - whatever values were in allocated memory space
x = torch.empty(5, 3)
print(x)
print()

# initialise with zeroes
x = torch.zeros(5, 3, dtype=torch.long)
print(x)
print()

# initialise with values
x = torch.tensor([5.5, 3])
print(x)
print()

# set new values
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

# set new values
x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)
print()

print(x.size())
# torch.Size supports tuple operations
print()

# numpy style indexing
print(x[:, 1])

# reshaping
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

# extract a scalar value
x = torch.randn(1)
print(x)
print(x.item())

In [ ]:
# random numpy array
array = np.random.rand(2,2)
print("{}\n {}\n".format(type(array),array))

# from numpy to tensor
from_numpy_to_tensor = torch.from_numpy(array)
print("{}\n".format(from_numpy_to_tensor))

# from tensor to numpy
from_tensor_to_numpy = from_numpy_to_tensor.numpy()
print("{}\n {}\n".format(type(from_tensor_to_numpy),from_tensor_to_numpy))
print()

# bridge to numpy - note that memory is shared
a = torch.ones(5)
print(a)
b = a.numpy()
print(b)
a.add_(1)
print(a)
print(b)

a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

In [ ]:
# create tensor 
tensor = torch.ones(3,3)
print("\n ",tensor)

# Resize with view
print("Resise:\n {} {} \n".format(tensor.view(9).shape, tensor.view(9)))

# Addition
print("Addition:\n {}\n".format(torch.add(tensor,tensor)))
print("Addition:\n {}\n".format(tensor.add(tensor)))
print("Addition:\n {}\n".format(tensor + tensor))

print('\nalso adding in place')
tensor.add_(tensor)
print(tensor)
tensor.add_(tensor)
print(tensor)
print()

# Subtraction
print("Subtraction:\n {}\n".format(torch.sub(tensor,tensor)))
print("Subtraction:\n {}\n".format(tensor.sub(tensor)))
print("Subtraction:\n {}\n".format(tensor - tensor))

# Element wise multiplication
print("Element wise multiplication:\n {}\n".format(torch.mul(tensor + tensor, tensor + tensor)))
print("Element wise multiplication:\n {}\n".format(tensor.mul(tensor + tensor)))
print("Element wise multiplication:\n {}\n".format((tensor + tensor) * (tensor + tensor)))

# Element wise division
print("Element wise division:\n {}\n".format(torch.div(tensor,tensor)))
print("Element wise division:\n {}\n".format(tensor.div(tensor)))
print("Element wise division:\n {}\n".format(tensor / (tensor + tensor)))

# Mean
tensor = torch.Tensor([[1,2,3,4,5], [6,7,8,9, 10]])
print("Mean:\n {}".format(tensor.mean(axis=1)))

# Standard deviation (std)
print("std:\n {}".format(tensor.std(axis=1)))

In [ ]:
# define variable - these are like tensors but optionally keep track of gradients
var = Variable(torch.ones(3), requires_grad = True)
var

In [ ]:
# quick look at backward propagation
input_values = torch.Tensor([2,3,4,5])
x = Variable(input_values, requires_grad = True)
print(" x =  ",x)
# perform some transformation on x
y = x**2
print(" y =  ",y)

# and a further calculation, e.g. a loss/cost function
o = sum((y - input_values)**2)/len(input_values)
print(" o =  ",o)

# backward
o.backward() # calculates gradients, which go back to x
print("x gradients: ", x.grad)

In [ ]:
# linear regression example

# sales regress on prices
n = 7
price_tensor = torch.Tensor([i+3 for i in range(n)]).view(-1,1)
price_tensor

# some random error (this sampling method creates a tensor of shape (1, n))
err = normal.Normal(torch.tensor([0.0]), torch.tensor([0.5]))
sales_tensor = torch.add(torch.Tensor([i+.5 for i in range(n,0,-1)]).view(-1,1),
                         err.sample(torch.tensor([n])))

# here's our linear relationship
plt.scatter(price_tensor,sales_tensor)
plt.xlabel("Price ($)")
plt.ylabel("Sales (1000's)")
plt.title("Price vs Sales")
plt.show()

In [ ]:
# create class - must have a forward function
# inherit everything from nn.Module
class LinearRegression(nn.Module):
    def __init__(self,input_size,output_size):
        super(LinearRegression, self).__init__()
        # Built in linear function
        self.linear = nn.Linear(input_dim, output_dim)
    # all models need a forward function defining the model structure
    # this one is simple
    def forward(self, x):
        return self.linear(x)
    
# define model
input_dim = 1
output_dim = 1
model = LinearRegression(input_dim, output_dim) # input and output size are 1

# MSE loss
mse = nn.MSELoss()

# Optimization
learning_rate = 0.02
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
# train model
n_iter = 1000
iloss = np.empty(n_iter)
for i in range(n_iter):
        
    # reset the gradients
    optimizer.zero_grad() 
    
    # Forward to get output - is there some kind of aliasing? forward is not explicitly called
    results = model(price_tensor)
    
    # Calculate Loss
    loss = mse(results, sales_tensor)
    
    # backward propagation - calculates gradients back through all the model parameters
    loss.backward()
    
    # Updating parameters
    optimizer.step()
    
    # info to UI
    iloss[i] = loss.data
    if(i % 50 == 0):
        print('epoch {}, loss {}'.format(i, loss.data))

plt.plot(range(n_iter),iloss)
plt.xlabel("Number of Iterations")
plt.ylabel("Loss")
plt.show()

In [ ]:
# predict our car price 
predicted = model(price_tensor)
plt.scatter(price_tensor,sales_tensor,
            label = "original data",color ="red")
plt.scatter(price_tensor,model(price_tensor).data,
            label = "predicted data",color ="blue")

plt.legend()
plt.xlabel("Car Price ($)")
plt.ylabel("Sales (1000's)")
plt.title("Original vs Predicted")
plt.show()

In [ ]:
# executing in the GPU
print(torch.cuda.is_available())

# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double)) 